# Data & Details

## Defulat folder setup

In [1]:
import os

# Change the current working directory
os.chdir('C:/Users/srigd/Desktop/Project/Python/General Details')

# Now, when you open a file without specifying a full path, Python looks in the current working directory
with open('another_file.txt', 'w') as f:
    f.write('This file is saved in the specified default directory.')

## Salesforce Connection

In [2]:
import requests
import pandas as pd
from simple_salesforce import Salesforce
import keyring

# Retrieve your credentials
username = keyring.get_password("salesforce", "username")
password = keyring.get_password("salesforce", "password")
security_token = keyring.get_password("salesforce", "security_token")
consumer_key = keyring.get_password("salesforce", "consumer_key")
consumer_secret = keyring.get_password("salesforce", "consumer_secret")

# Step 1: Obtain OAuth 2.0 token
token_url = "https://login.salesforce.com/services/oauth2/token"
payload = {
    'grant_type': 'password',
    'client_id': consumer_key,
    'client_secret': consumer_secret,
    'username': username,
    'password': password + security_token
}
response = requests.post(token_url, data=payload)
response.raise_for_status()  # Check if the request was successful

# Extract access token from the response
access_token = response.json().get('access_token')
instance_url = response.json().get('instance_url')

# Step 2: Authenticate to Salesforce using the access token
sf = Salesforce(instance_url=instance_url, session_id=access_token)

## Metadata extractor

In [3]:
def get_salesforce_metadata(sf):
    metadata = []
    # Get all objects
    objects = sf.describe()["sobjects"]
    
    for obj in objects:
        obj_name = obj["name"]
        fields = sf.__getattr__(obj_name).describe()["fields"]
        
        for field in fields:
            field_metadata = {
                "object": obj_name,
                "field": field["name"],
                "type": field["type"],
                "length": field.get("length"),
                "relationship": field.get("relationshipName"),
                "related_to": field.get("referenceTo")
            }
            metadata.append(field_metadata)
    
    return pd.DataFrame(metadata)

metadata_df = get_salesforce_metadata(sf)


In [5]:
metadata_df.to_excel('metadata_df.xlsx')

## Calculate Data Completeness

In [15]:
from simple_salesforce import Salesforce, SalesforceMalformedRequest
import pandas as pd
import time

def calculate_data_completeness(sf, metadata_df, max_fields_per_object=10):
    completeness = []
    
    for obj in metadata_df["object"].unique():
        start_time = time.time()
        try:
            query = f"SELECT COUNT() FROM {obj}"
            total_records = sf.query(query)["totalSize"]
        except (SalesforceMalformedRequest, Exception) as e:
            # Skip objects that do not support querying
            print(f"Skipping {obj} due to error: {e}")
            continue
        
        fields_processed = 0
        for field in metadata_df[metadata_df["object"] == obj]["field"]:
            if fields_processed >= max_fields_per_object:
                break
            try:
                # Use a different approach to count non-null values
                query = f"SELECT {field} FROM {obj} WHERE {field} != null"
                filled_records = sf.query(query)["totalSize"]
                
                completeness.append({
                    "object": obj,
                    "field": field,
                    "total_records": total_records,
                    "filled_records": filled_records,
                    "completeness_percentage": (filled_records / total_records) * 100 if total_records else 0
                })
                fields_processed += 1
                print(f"Processed field {field} in object {obj}")
            except (SalesforceMalformedRequest, Exception) as e:
                # Skip fields that cause errors
                print(f"Skipping field {field} in object {obj} due to error: {e}")
                continue
        
        end_time = time.time()
        print(f"Processed object {obj} in {end_time - start_time:.2f} seconds")

    return pd.DataFrame(completeness)

# Assuming you have already connected to Salesforce and retrieved metadata_df
completeness_df = calculate_data_completeness(sf, metadata_df, max_fields_per_object=5)

# import ace_tools as tools
# tools.display_dataframe_to_user(name="Data Completeness", dataframe=completeness_df)


Skipping ACTIVITES_EMv2__ChangeEvent due to error: Malformed request https://my-edwardmellor.my.salesforce.com/services/data/v59.0/query/?q=SELECT+COUNT%28%29+FROM+ACTIVITES_EMv2__ChangeEvent. Response content: [{'message': 'entity type ACTIVITES_EMv2__ChangeEvent does not support query', 'errorCode': 'INVALID_TYPE_FOR_OPERATION'}]
Processed field Id in object ACTIVITES_EMv2__History
Processed field IsDeleted in object ACTIVITES_EMv2__History
Processed field ParentId in object ACTIVITES_EMv2__History
Processed field CreatedById in object ACTIVITES_EMv2__History
Processed field CreatedDate in object ACTIVITES_EMv2__History
Processed object ACTIVITES_EMv2__History in 0.72 seconds
Processed field Id in object ACTIVITES_EMv2__c
Processed field IsDeleted in object ACTIVITES_EMv2__c
Processed field Name in object ACTIVITES_EMv2__c
Processed field CurrencyIsoCode in object ACTIVITES_EMv2__c
Processed field CreatedDate in object ACTIVITES_EMv2__c
Processed object ACTIVITES_EMv2__c in 0.75 seco

ModuleNotFoundError: No module named 'ace_tools'

In [30]:
completeness_df.to_excel('completeness_df.xlsx')

## Analyze Relationships

In [8]:
def analyze_relationships(metadata_df):
    relationships = metadata_df.dropna(subset=["relationship"]).copy()
    relationships["connection"] = relationships.apply(lambda row: f"{row['object']} -> {row['related_to']}", axis=1)
    return relationships[["object", "field", "type", "relationship", "related_to", "connection"]]

relationships_df = analyze_relationships(metadata_df)
relationships_df


,object,field,type,relationship,related_to,connection
45,ACTIVITES_EMv2__History,ParentId,reference,Parent,[ACTIVITES_EMv2__c],ACTIVITES_EMv2__History -> ['ACTIVITES_EMv2__c']
46,ACTIVITES_EMv2__History,CreatedById,reference,CreatedBy,[User],ACTIVITES_EMv2__History -> ['User']
57,ACTIVITES_EMv2__c,CreatedById,reference,CreatedBy,[User],ACTIVITES_EMv2__c -> ['User']
59,ACTIVITES_EMv2__c,LastModifiedById,reference,LastModifiedBy,[User],ACTIVITES_EMv2__c -> ['User']
61,ACTIVITES_EMv2__c,ACTIVITY_Type__c,reference,ACTIVITY_Type__r,[ACTIVITY_Template_EMv2__c],ACTIVITES_EMv2__c -> ['ACTIVITY_Template_EMv2_...
...,...,...,...,...,...,...
44380,timeline__Timeline_Child_Object__c,LastModifiedById,reference,LastModifiedBy,[User],timeline__Timeline_Child_Object__c -> ['User']
44382,timeline__Timeline_Child_Object__c,timeline__Timeline_Configuration__c,reference,timeline__Timeline_Configuration__r,[timeline__Timeline_Configuration__c],timeline__Timeline_Child_Object__c -> ['timeli...
44418,timeline__Timeline_Configuration__c,OwnerId,reference,Owner,"[Group, User]",timeline__Timeline_Configuration__c -> ['Group...
44423,timeline__Timeline_Configuration__c,CreatedById,reference,CreatedBy,[User],timeline__Timeline_Configuration__c -> ['User']


In [9]:
relationships_df.to_excel('relationships_df.xlsx')

## Query Producer from report

In [6]:
from simple_salesforce import Salesforce
import json
import os

def get_report_soql(sf, report_id):
    # Describe the report to get its details
    report_describe = sf.restful(f'analytics/reports/{report_id}/describe')
    
    # Extract the report metadata
    report_metadata = report_describe['reportMetadata']
    
    # Build the SOQL query
    select_clause = ', '.join(report_metadata['detailColumns'])
    from_clause = report_metadata['reportType']['type']
    
    # Handle the reportFilters more robustly
    where_clauses = []
    for filter_item in report_metadata['reportFilters']:
        if isinstance(filter_item, dict):
            column = filter_item.get('column')
            operator = filter_item.get('operator')
            value = filter_item.get('value')
            if column and operator:
                where_clause = f"{column} {operator} '{value}'" if value else f"{column} {operator}"
                where_clauses.append(where_clause)
    
    where_clause = ' AND '.join(where_clauses)
    
    soql = f"SELECT {select_clause} FROM {from_clause}"
    if where_clause:
        soql += f" WHERE {where_clause}"
    
    return soql

# Salesforce login credentials
sf_username = os.environ.get('SF_USERNAME')
sf_password = os.environ.get('SF_PASSWORD')
sf_security_token = os.environ.get('SF_SECURITY_TOKEN')

# Initialize Salesforce connection
sf = Salesforce(username=username, password=password, security_token=security_token)

# Specify the report ID
report_id = '00O8W0000071qebUAA'  # Your actual report ID

# Get the SOQL for the report
soql = get_report_soql(sf, report_id)

print(f"SOQL for report {report_id}:")
print(soql)

SOQL for report 00O8W0000071qebUAA:
SELECT CUST_NAME, FK_NAME, CUST_CREATED_DATE, pba__Closing__c.pba__Listing__c, pba__Closing__c.Closing_Status__c, FK_pba__Offer__c.pba__Status__c, pba__Closing__c.Calculated_Fee__c, FK_pba__Offer__c.Source_Of_Funding__c FROM CustomEntity$pba__Closing__c@pba__Closing__c.pba__Offer__c WHERE CUST_RECORDTYPE equals 'Sale' AND pba__Closing__c.Office__c notEqual 'Auction'


## Query Producer from report (task & Event)

In [25]:
from simple_salesforce import Salesforce
import json
import os

def get_report_soql(sf, report_id):
    # Describe the report to get its details
    report_describe = sf.restful(f'analytics/reports/{report_id}/describe')
    
    # Extract the report metadata
    report_metadata = report_describe['reportMetadata']
    
    # Build the SOQL query
    select_clause = ', '.join(report_metadata['detailColumns'])
    from_clause = report_metadata['reportType']['type']
    
    # Handle the reportFilters more robustly
    where_clauses = []
    for filter_item in report_metadata['reportFilters']:
        if isinstance(filter_item, dict):
            column = filter_item.get('column')
            operator = filter_item.get('operator')
            value = filter_item.get('value')
            if column and operator:
                where_clause = f"{column} {operator} '{value}'" if value else f"{column} {operator}"
                where_clauses.append(where_clause)
    
    where_clause = ' AND '.join(where_clauses)
    
    # Check if the report is on activities (Events and Tasks)
    if 'Activity' in from_clause:
        # Create separate queries for Events and Tasks
        event_soql = f"SELECT {select_clause} FROM Event"
        task_soql = f"SELECT {select_clause} FROM Task"
        
        if where_clause:
            event_soql += f" WHERE {where_clause}"
            task_soql += f" WHERE {where_clause}"
        
        # Combine the queries with UNION
        soql = f"({event_soql}) UNION ({task_soql})"
    else:
        # For non-activity reports, use the original logic
        soql = f"SELECT {select_clause} FROM {from_clause}"
        if where_clause:
            soql += f" WHERE {where_clause}"
    
    return soql

# Salesforce login credentials
sf_username = os.environ.get('SF_USERNAME')
sf_password = os.environ.get('SF_PASSWORD')
sf_security_token = os.environ.get('SF_SECURITY_TOKEN')

# Initialize Salesforce connection
sf = Salesforce(username=username, password=password, security_token=security_token)

# Specify the report ID
report_id = '00O8W0000071qebUAA'  # Your actual report ID

# Get the SOQL for the report
soql = get_report_soql(sf, report_id)

print(f"SOQL for report {report_id}:")
print(soql)

SOQL for report 00O8W0000071qebUAA:
SELECT CUST_NAME, FK_NAME, CUST_CREATED_DATE, pba__Closing__c.pba__Listing__c, pba__Closing__c.Closing_Status__c, FK_pba__Offer__c.pba__Status__c, pba__Closing__c.Calculated_Fee__c, FK_pba__Offer__c.Source_Of_Funding__c FROM CustomEntity$pba__Closing__c@pba__Closing__c.pba__Offer__c WHERE CUST_RECORDTYPE equals 'Sale' AND pba__Closing__c.Office__c notEqual 'Auction'


## Object & Columns & population

In [33]:
import requests
import pandas as pd
from simple_salesforce import Salesforce
import keyring
from concurrent.futures import ThreadPoolExecutor, as_completed

def connect_to_salesforce():
    username = keyring.get_password("salesforce", "username")
    password = keyring.get_password("salesforce", "password")
    security_token = keyring.get_password("salesforce", "security_token")
    consumer_key = keyring.get_password("salesforce", "consumer_key")
    consumer_secret = keyring.get_password("salesforce", "consumer_secret")

    token_url = "https://login.salesforce.com/services/oauth2/token"
    payload = {
        'grant_type': 'password',
        'client_id': consumer_key,
        'client_secret': consumer_secret,
        'username': username,
        'password': password + security_token
    }
    response = requests.post(token_url, data=payload)
    response.raise_for_status()

    access_token = response.json().get('access_token')
    instance_url = response.json().get('instance_url')

    return Salesforce(instance_url=instance_url, session_id=access_token)

def analyze_object(sf, obj_name):
    try:
        describe = getattr(sf, obj_name).describe()
        fields = describe['fields']
        
        results = []
        for field in fields:
            field_name = field['name']
            total_records = sf.query(f"SELECT COUNT() FROM {obj_name}")['totalSize']
            missing_records = sf.query(f"SELECT COUNT() FROM {obj_name} WHERE {field_name} = NULL")['totalSize']
            
            results.append({
                'object': obj_name,
                'field': field_name,
                'total_records': total_records,
                'missing_records': missing_records
            })
        
        return results
    except Exception as e:
        return [{
            'object': obj_name,
            'field': 'ERROR',
            'total_records': 0,
            'missing_records': 0,
            'error': str(e)
        }]

def analyze_salesforce_objects():
    sf = connect_to_salesforce()
    
    describe_result = sf.describe()
    all_objects = [obj['name'] for obj in describe_result['sobjects']]
    
    results = []
    with ThreadPoolExecutor(max_workers=10) as executor:
        future_to_obj = {executor.submit(analyze_object, sf, obj): obj for obj in all_objects}
        for future in as_completed(future_to_obj):
            results.extend(future.result())
    
    df = pd.DataFrame(results)
    df['missing_percentage'] = (df['missing_records'] / df['total_records']) * 100
    df = df.sort_values(['object', 'missing_percentage'], ascending=[True, False])
    
    df.to_csv('salesforce_detailed_object_analysis.csv', index=False)
    print("Analysis complete. Results saved to 'salesforce_detailed_object_analysis.csv'")

if __name__ == "__main__":
    analyze_salesforce_objects()

Analysis complete. Results saved to 'salesforce_detailed_object_analysis.csv'


## How to connect two tables

In [5]:
import requests
from simple_salesforce import Salesforce
import keyring

def get_salesforce_session():
    # Retrieve your credentials
    username = keyring.get_password("salesforce", "username")
    password = keyring.get_password("salesforce", "password")
    security_token = keyring.get_password("salesforce", "security_token")
    consumer_key = keyring.get_password("salesforce", "consumer_key")
    consumer_secret = keyring.get_password("salesforce", "consumer_secret")

    # Obtain OAuth 2.0 token
    token_url = "https://login.salesforce.com/services/oauth2/token"
    payload = {
        'grant_type': 'password',
        'client_id': consumer_key,
        'client_secret': consumer_secret,
        'username': username,
        'password': password + security_token
    }
    response = requests.post(token_url, data=payload)
    response.raise_for_status()  # Check if the request was successful

    # Extract access token from the response
    access_token = response.json().get('access_token')
    instance_url = response.json().get('instance_url')

    # Authenticate to Salesforce using the access token
    sf = Salesforce(instance_url=instance_url, session_id=access_token)
    return sf

def get_object_fields(sf, object_name):
    object_metadata = sf.__getattr__(object_name).describe()
    fields = {field['name']: field for field in object_metadata['fields']}
    return fields

def find_common_fields(object1_fields, object2_fields):
    common_fields = set(object1_fields.keys()).intersection(set(object2_fields.keys()))
    return common_fields

def find_reference_fields(object_fields, reference_to_object):
    reference_fields = [field['name'] for field in object_fields.values()
                        if 'referenceTo' in field and reference_to_object in field['referenceTo']]
    return reference_fields

def get_joinable_fields(object1, object2):
    sf = get_salesforce_session()

    # Get fields for both objects
    object1_fields = get_object_fields(sf, object1)
    object2_fields = get_object_fields(sf, object2)

    # Find common fields by name
    common_fields = find_common_fields(object1_fields, object2_fields)

    # Find reference fields
    object1_references_to_object2 = find_reference_fields(object1_fields, object2)
    object2_references_to_object1 = find_reference_fields(object2_fields, object1)

    return {
        'common_fields': list(common_fields),
        'object1_references_to_object2': object1_references_to_object2,
        'object2_references_to_object1': object2_references_to_object1
    }

# Example usage
object1 = 'pba__Closing__c'
object2 = 'pba__Listing__c'
joinable_fields = get_joinable_fields(object1, object2)
print(f"Common Fields: {joinable_fields['common_fields']}")
print(f"{object1} references {object2} on fields: {joinable_fields['object1_references_to_object2']}")
print(f"{object2} references {object1} on fields: {joinable_fields['object2_references_to_object1']}")


Common Fields: ['LastModifiedDate', 'Joint_Agency_Calculated_Fee__c', 'Organisation_Domain__c', 'MB_logo__c', 'Current_User_Email__c', 'EPC__c', 'OwnerId', 'LastModifiedById', 'Rewards_Lister__c', 'Calculated_Fee__c', 'Name', 'Current_User_Name__c', 'IsDeleted', 'Vendor_Manager__c', 'Lister_Name_Formula__c', 'Pod_Match_Formula__c', 'MB_Property_Ombudsman_logo__c', 'LastViewedDate', 'RecordTypeId', 'Property_Owner_Household__c', 'Record_Type_Name__c', 'Current_User_Phone__c', 'LastActivityDate', 'Office__c', 'Exchange_Fee__c', 'CreatedById', 'CreatedDate', 'SystemModstamp', 'Pod__c', 'Lister_Name__c', 'CurrencyIsoCode', 'EM_logo__c', 'LastReferencedDate', 'Todays_date__c', 'EM_Openworks_Property_Ombudsman_logo__c', 'Entry_Fee__c', 'Solicitor_Referred_By_Who__c', 'Id', 'Success_Fee_To_Be_Paid__c']
pba__Closing__c references pba__Listing__c on fields: ['pba__Listing__c']
pba__Listing__c references pba__Closing__c on fields: []


In [4]:
import requests
import pandas as pd
from simple_salesforce import Salesforce
import keyring

def get_salesforce_session():
    # Retrieve your credentials
    username = keyring.get_password("salesforce", "username")
    password = keyring.get_password("salesforce", "password")
    security_token = keyring.get_password("salesforce", "security_token")
    consumer_key = keyring.get_password("salesforce", "consumer_key")
    consumer_secret = keyring.get_password("salesforce", "consumer_secret")

    # Obtain OAuth 2.0 token
    token_url = "https://login.salesforce.com/services/oauth2/token"
    payload = {
        'grant_type': 'password',
        'client_id': consumer_key,
        'client_secret': consumer_secret,
        'username': username,
        'password': password + security_token
    }
    response = requests.post(token_url, data=payload)
    response.raise_for_status()  # Check if the request was successful

    # Extract access token from the response
    access_token = response.json().get('access_token')
    instance_url = response.json().get('instance_url')

    # Authenticate to Salesforce using the access token
    sf = Salesforce(instance_url=instance_url, session_id=access_token)
    return sf

def get_object_fields(sf, object_name):
    object_metadata = sf.__getattr__(object_name).describe()
    fields = {field['name']: field for field in object_metadata['fields']}
    return fields

def find_common_fields(object1_fields, object2_fields):
    common_fields = set(object1_fields.keys()).intersection(set(object2_fields.keys()))
    return list(common_fields)

def find_reference_fields(object_fields, reference_to_object):
    reference_fields = [field['name'] for field in object_fields.values()
                        if 'referenceTo' in field and reference_to_object in field['referenceTo']]
    return reference_fields

def get_joinable_fields(object1, object2):
    sf = get_salesforce_session()

    # Get fields for both objects
    object1_fields = get_object_fields(sf, object1)
    object2_fields = get_object_fields(sf, object2)

    # Find common fields by name
    common_fields = find_common_fields(object1_fields, object2_fields)

    # Find reference fields
    object1_references_to_object2 = find_reference_fields(object1_fields, object2)
    object2_references_to_object1 = find_reference_fields(object2_fields, object1)

    # Create DataFrame
    data = {
        'Common Fields': common_fields,
        f'{object1} References {object2}': object1_references_to_object2,
        f'{object2} References {object1}': object2_references_to_object1
    }

    # Since the length of lists can differ, creating DataFrame in this way might lead to NaN values for mismatched lengths
    max_len = max(len(common_fields), len(object1_references_to_object2), len(object2_references_to_object1))
    for key in data:
        data[key] += [None] * (max_len - len(data[key]))  # Fill shorter lists with None

    df = pd.DataFrame(data)
    return df

# Example usage
object1 = 'pba__Closing__c'
object2 = 'pba__Listing__c'
joinable_fields_df = get_joinable_fields(object1, object2)
joinable_fields_df


,Common Fields,pba__Closing__c References pba__Listing__c,pba__Listing__c References pba__Closing__c
0,LastModifiedDate,pba__Listing__c,None
1,Joint_Agency_Calculated_Fee__c,None,None
2,Organisation_Domain__c,None,None
3,MB_logo__c,None,None
4,Current_User_Email__c,None,None
5,EPC__c,None,None
6,OwnerId,None,None
7,LastModifiedById,None,None
8,Rewards_Lister__c,None,None
9,Calculated_Fee__c,None,None


### Formula Finder

In [11]:
import requests
import keyring

def get_salesforce_formula(object_name, field_name):
    # Retrieve your credentials
    username = keyring.get_password("salesforce", "username")
    password = keyring.get_password("salesforce", "password")
    security_token = keyring.get_password("salesforce", "security_token")
    consumer_key = keyring.get_password("salesforce", "consumer_key")
    consumer_secret = keyring.get_password("salesforce", "consumer_secret")

    # Step 1: Obtain OAuth 2.0 token
    token_url = "https://login.salesforce.com/services/oauth2/token"
    payload = {
        'grant_type': 'password',
        'client_id': consumer_key,
        'client_secret': consumer_secret,
        'username': username,
        'password': password + security_token
    }
    response = requests.post(token_url, data=payload)
    response.raise_for_status()  # Check if the request was successful

    # Extract access token from the response
    access_token = response.json().get('access_token')
    instance_url = response.json().get('instance_url')

    # Step 2: Retrieve object metadata
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/json'
    }
    metadata_url = f"{instance_url}/services/data/v52.0/sobjects/{object_name}/describe"
    metadata_response = requests.get(metadata_url, headers=headers)
    metadata_response.raise_for_status()  # Check if the request was successful
    metadata = metadata_response.json()

    # Step 3: Find the specific field and extract its formula
    formula = None
    for field in metadata['fields']:
        if field['name'] == field_name:
            if field.get('calculated') and field.get('calculatedFormula'):
                formula = field['calculatedFormula']
            else:
                formula = "The specified field is not a calculated field."
            break
    else:
        formula = "Field not found in the specified object."

    return formula

# Example usage
object_name = 'OPPORTUNITY'
field_name = 'OPPORTUNITY_NAME'
formula = get_salesforce_formula(object_name, field_name)
print(f"Field: {field_name}, Formula: {formula}")


Field: OPPORTUNITY_NAME, Formula: Field not found in the specified object.


### Get the table or sample

In [14]:
import requests
import pandas as pd
from simple_salesforce import Salesforce
import keyring
import sweetviz

def export_salesforce_data(object_name, record_limit=None):
    # Retrieve your credentials
    username = keyring.get_password("salesforce", "username")
    password = keyring.get_password("salesforce", "password")
    security_token = keyring.get_password("salesforce", "security_token")
    consumer_key = keyring.get_password("salesforce", "consumer_key")
    consumer_secret = keyring.get_password("salesforce", "consumer_secret")

    # Step 1: Obtain OAuth 2.0 token
    token_url = "https://login.salesforce.com/services/oauth2/token"
    payload = {
        'grant_type': 'password',
        'client_id': consumer_key,
        'client_secret': consumer_secret,
        'username': username,
        'password': password + security_token
    }
    response = requests.post(token_url, data=payload)
    response.raise_for_status()  # Check if the request was successful

    # Extract access token from the response
    access_token = response.json().get('access_token')
    instance_url = response.json().get('instance_url')

    # Step 2: Authenticate to Salesforce using the access token
    sf = Salesforce(instance_url=instance_url, session_id=access_token)

    # Step 3: Retrieve object metadata to get all column names
    object_metadata = sf.__getattr__(object_name).describe()
    column_names = [field['name'] for field in object_metadata['fields']]

    # Construct SOQL query to get all data from the object
    query = f"SELECT {', '.join(column_names)} FROM {object_name}"
    if record_limit:
        query += f" LIMIT {record_limit}"

    # Step 4: Query data
    records = []
    query_result = sf.query_all(query)
    records.extend(query_result['records'])

    # Continue querying if there are more records and no limit was set
    while not query_result['done'] and not record_limit:
        query_result = sf.query_more(query_result['nextRecordsUrl'], True)
        records.extend(query_result['records'])

    # Convert the data to a pandas DataFrame
    df = pd.DataFrame(records)

    # Clean up the DataFrame (remove Salesforce metadata)
    if 'attributes' in df.columns:
        df = df.drop(columns=['attributes'])

    return df

# Example usage
object_name = 'pba__Offer__c'
#record_limit = 5000  # Set to None to retrieve all records
df = export_salesforce_data(object_name, record_limit=None)
df

# Generate Sweetviz report
# report = sweetviz.analyze(df)
# report.show_html('report.html')

# You can now analyze the DataFrame df as needed
# print(df.head())


,Id,OwnerId,IsDeleted,Name,CurrencyIsoCode,RecordTypeId,CreatedDate,CreatedById,LastModifiedDate,LastModifiedById,...,Offer_Notes_2__c,Offer_Notes_3__c,Offer_Notes_4__c,Offer_Notes_5__c,Offer_Notes_6__c,Offer_Notes_7__c,Offer_Notes_8__c,Offer_Notes_9__c,Offer_Notes_10__c,Organisation_Domain__c
0,a0G3t00000HCjlzEAD,0053t000006YrbuAAC,False,O-003209,GBP,0123t000000NWcbAAG,2021-02-16T12:11:48.000+0000,0053t000006YrbuAAC,2024-05-21T09:14:55.000+0000,0053t000008Udn5AAC,...,None,None,None,None,None,None,None,None,None,None
1,a0G3t00000HCl3KEAT,0053t000008UnS3AAK,False,O-007128,GBP,0123t000000NWcbAAG,2021-02-15T16:46:00.000+0000,0053t000008UbxuAAC,2021-02-16T17:43:55.000+0000,0053t000008UbxuAAC,...,None,None,None,None,None,None,None,None,None,None
2,a0G3t00000HCl3LEAT,0053t000008UnRzAAK,False,O-007129,GBP,0123t000000NWcbAAG,2021-02-15T16:28:00.000+0000,0053t000008UbxuAAC,2021-02-16T17:43:55.000+0000,0053t000008UbxuAAC,...,None,None,None,None,None,None,None,None,None,None
3,a0G3t00000HCl3MEAT,0053t000008UnRUAA0,False,O-007130,GBP,0123t000000NWcbAAG,2021-02-15T16:16:00.000+0000,0053t000008UbxuAAC,2021-02-16T17:43:55.000+0000,0053t000008UbxuAAC,...,None,None,None,None,None,None,None,None,None,None
4,a0G3t00000HCl3NEAT,0053t000008UnRUAA0,False,O-007131,GBP,0123t000000NWcbAAG,2021-02-15T14:45:00.000+0000,0053t000008UbxuAAC,2021-02-16T17:43:55.000+0000,0053t000008UbxuAAC,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34892,a0GWS000000UnHp2AK,0053t000008UnRyAAK,False,O-050094,GBP,0123t000000GDXVAA4,2024-07-24T16:15:24.000+0000,0053t000008t8wgAAA,2024-07-24T16:16:07.000+0000,0053t000008t8wgAAA,...,None,None,None,None,None,None,None,None,None,None
34893,a0GWS000000UnuX2AS,0053t000008t2DCAAY,False,O-050095,GBP,0123t000000NWcbAAG,2024-07-25T07:33:53.000+0000,0053t000008t2DCAAY,2024-07-25T07:35:15.000+0000,0053t000008t2DCAAY,...,None,None,None,None,None,None,None,None,None,None
34894,a0GWS000000Uo0z2AC,0058W00000CWfZ9QAL,False,O-050096,GBP,0123t000000NWcbAAG,2024-07-25T08:55:29.000+0000,0058W00000CWfZ9QAL,2024-07-25T10:29:13.000+0000,0053t000008UnRMAA0,...,None,None,None,None,None,None,None,None,None,None
34895,a0GWS000000UoNZ2A0,0058W00000FilU7QAJ,False,O-050097,GBP,0123t000000Akl8AAC,2024-07-25T09:54:35.000+0000,0058W00000FilU7QAJ,2024-07-25T09:55:10.000+0000,0058W00000FilU7QAJ,...,None,None,None,None,None,None,None,None,None,None


In [ ]:
df.to_csv('Offer.csv')

## Table extractor with query

In [3]:
import requests
import pandas as pd
from simple_salesforce import Salesforce
import keyring

def export_salesforce_data(query, export_file):
    # Retrieve your credentials
    username = keyring.get_password("salesforce", "username")
    password = keyring.get_password("salesforce", "password")
    security_token = keyring.get_password("salesforce", "security_token")
    consumer_key = keyring.get_password("salesforce", "consumer_key")
    consumer_secret = keyring.get_password("salesforce", "consumer_secret")

    # Step 1: Obtain OAuth 2.0 token
    token_url = "https://login.salesforce.com/services/oauth2/token"
    payload = {
        'grant_type': 'password',
        'client_id': consumer_key,
        'client_secret': consumer_secret,
        'username': username,
        'password': password + security_token
    }
    response = requests.post(token_url, data=payload)
    response.raise_for_status()  # Check if the request was successful

    # Extract access token from the response
    access_token = response.json().get('access_token')
    instance_url = response.json().get('instance_url')

    # Step 2: Authenticate to Salesforce using the access token
    sf = Salesforce(instance_url=instance_url, session_id=access_token)

    # Step 3: Query data
    records = []
    query_result = sf.query_all(query)
    records.extend(query_result['records'])

    # Continue querying if there are more records
    while not query_result['done']:
        query_result = sf.query_more(query_result['nextRecordsUrl'], True)
        records.extend(query_result['records'])

    # Convert the data to a pandas DataFrame
    df = pd.DataFrame(records)

    # Clean up the DataFrame (remove Salesforce metadata)
    if 'attributes' in df.columns:
        df = df.drop(columns=['attributes'])

    # Export the DataFrame to a CSV file
    df.to_csv(export_file, index=False)
    print(f"Data exported to {export_file}")

# Example usage
query = """
SELECT 




User__c, 
Name, 
Primary__r.Name,
Role_Title__r.Name  






from Staff__c
"""
export_file = 'test.csv'

export_salesforce_data(query, export_file)


Data exported to test.csv


## Table extractor or sample

In [17]:
import requests
import pandas as pd
from simple_salesforce import Salesforce
import keyring
import sweetviz

def export_salesforce_data(object_name, record_limit=None):
    # Retrieve your credentials
    username = keyring.get_password("salesforce", "username")
    password = keyring.get_password("salesforce", "password")
    security_token = keyring.get_password("salesforce", "security_token")
    consumer_key = keyring.get_password("salesforce", "consumer_key")
    consumer_secret = keyring.get_password("salesforce", "consumer_secret")

    # Step 1: Obtain OAuth 2.0 token
    token_url = "https://login.salesforce.com/services/oauth2/token"
    payload = {
        'grant_type': 'password',
        'client_id': consumer_key,
        'client_secret': consumer_secret,
        'username': username,
        'password': password + security_token
    }
    response = requests.post(token_url, data=payload)
    response.raise_for_status()  # Check if the request was successful

    # Extract access token from the response
    access_token = response.json().get('access_token')
    instance_url = response.json().get('instance_url')

    # Step 2: Authenticate to Salesforce using the access token
    sf = Salesforce(instance_url=instance_url, session_id=access_token)

    # Step 3: Retrieve object metadata to get all column names
    object_metadata = sf.__getattr__(object_name).describe()
    column_names = [field['name'] for field in object_metadata['fields']]

    # Construct SOQL query to get all data from the object
    query = f"SELECT {', '.join(column_names)} FROM {object_name}"
    if record_limit:
        query += f" LIMIT {record_limit}"

    # Step 4: Query data
    records = []
    query_result = sf.query_all(query)
    records.extend(query_result['records'])

    # Continue querying if there are more records and no limit was set
    while not query_result['done'] and not record_limit:
        query_result = sf.query_more(query_result['nextRecordsUrl'], True)
        records.extend(query_result['records'])

    # Convert the data to a pandas DataFrame
    df = pd.DataFrame(records)

    # Clean up the DataFrame (remove Salesforce metadata)
    if 'attributes' in df.columns:
        df = df.drop(columns=['attributes'])

    return df

# Example usage
object_name = 'Staff__c'
#record_limit = 5000  # Set to None to retrieve all records
df = export_salesforce_data(object_name, record_limit=None)
df

# Generate Sweetviz report
# report = sweetviz.analyze(df)
# report.show_html('report.html')

# You can now analyze the DataFrame df as needed
# print(df.head())


,Id,IsDeleted,Name,CurrencyIsoCode,CreatedDate,CreatedById,LastModifiedDate,LastModifiedById,SystemModstamp,LastViewedDate,...,VCCSSOExtID__c,VCCIn_Country_Display__c,VCCUsername__c,VCC_NvmAgentID__c,VCCCallback_Numbers__c,VCCName__c,VCC_Divert_Number__c,JANUS_Email_all_Updates__c,Office_Name__c,POD_Name__c
0,a3o8W000000jmbxQAA,False,Mehak Nazam,GBP,2023-10-19T14:16:26.000+0000,0058W00000BZQ4dQAH,2024-08-08T15:51:19.000+0000,0058W00000BZQ4dQAH,2024-08-08T15:51:19.000+0000,None,...,None,National,mehak.nazam@edwardmellor.co.uk,1086,None,Mehak Nazam,None,False,Financial Services,Financial Services
1,a3o8W000000jmc2QAA,False,Lindsay Heath,GBP,2023-10-19T16:21:26.000+0000,0053t000008Udn5AAC,2024-08-08T15:51:19.000+0000,0058W00000BZQ4dQAH,2024-08-08T15:51:19.000+0000,None,...,None,National,lindsay.heath@edwardmellor.co.uk,1079,None,Lindsay Heath,None,False,Financial Services,Financial Services
2,a3o8W000000jmc7QAA,False,Brooke Rushworth,GBP,2023-10-25T14:56:15.000+0000,0053t000008Udn5AAC,2023-10-25T14:56:15.000+0000,0053t000008Udn5AAC,2023-10-25T14:56:15.000+0000,None,...,None,None,brookejaydenrushworth@gmail.com,None,None,Brooke Rushworth,None,False,To Be Completed,None
3,a3o8W000000jmcRQAQ,False,Christian Vose,GBP,2023-10-31T17:01:44.000+0000,0053t000008Udn5AAC,2024-08-08T15:51:19.000+0000,0058W00000BZQ4dQAH,2024-08-08T15:51:19.000+0000,None,...,None,National,christian.vose@edwardmellor.co.uk,1034,None,Christian Vose,None,False,Droylsden,Droylsden & Gorton
4,a3o8W000000jmcWQAQ,False,Issabella Chapman,GBP,2023-11-01T15:24:04.000+0000,0053t000008Udn5AAC,2023-11-22T16:55:27.000+0000,0058W00000BZQ4dQAH,2023-12-08T07:31:04.000+0000,None,...,None,National,issabella.chapman@edwardmellor.co.uk,1066,None,Issabella Chapman,None,False,Auction,Auction
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,a3o8W000001vq4zQAA,False,Deepankar Srigyan,GBP,2024-07-15T09:11:31.000+0000,0058W00000BZQ4dQAH,2024-08-08T15:51:19.000+0000,0058W00000BZQ4dQAH,2024-08-08T15:51:19.000+0000,None,...,None,None,deepankar.srigyan@edwardmellor.co.uk,1141,441614434511,Deepankar Srigyan,None,False,Head Office,Head Office
158,a3o8W000001vq54QAA,False,Grace Beaumont,GBP,2024-07-16T14:41:01.000+0000,0058W00000BZQ4dQAH,2024-08-08T15:51:19.000+0000,0058W00000BZQ4dQAH,2024-08-08T15:51:19.000+0000,None,...,None,National,grace.beaumont@edwardmellor.co.uk,1142,None,Grace Beaumont,None,False,Head Office,Head Office
159,a3oWS0000007UfhYAE,False,Catherine Dixon,GBP,2024-07-23T10:49:29.000+0000,0058W00000BZQ4dQAH,2024-07-23T13:12:07.000+0000,0058W00000BZQ4dQAH,2024-07-23T13:12:07.000+0000,None,...,None,None,None,1136,None,Catherine Dixon,+447935115340,False,Head Office,Head Office
160,a3oWS0000007VFBYA2,False,Stephen Hankey,GBP,2024-07-25T11:23:59.000+0000,0053t000008Udn5AAC,2024-08-08T15:51:19.000+0000,0058W00000BZQ4dQAH,2024-08-08T15:51:19.000+0000,None,...,None,None,stephenjh.71@gmail.com,None,None,Stephen Hankey,None,False,Marple,Woodley & Marple


In [20]:
df.to_csv('Offer.csv')

### dataset profile

In [9]:
import dtale
import sweetviz

In [12]:
df.dtypes

Id                                                  object
Username                                            object
LastName                                            object
FirstName                                           object
Name                                                object
                                                     ...  
NVMContactWorld__MostRecentCallEventTimestamp__c    object
NVMContactWorld__MostRecentCallIsActive__c            bool
NVMContactWorld__MostRecentCall__c                  object
NVMContactWorld__NVM_Agent_Id__c                    object
NVMContactWorld__NotesCollapsed__c                    bool
Length: 254, dtype: object

In [15]:
def convert_complex_to_string(value):
    if isinstance(value, (dict, list, set)):
        return str(value)
    return value

# Apply the function to the entire DataFrame
df = df.map(convert_complex_to_string)

In [21]:
print(pd.__version__)

2.2.2


In [22]:
user_report  = sweetviz.analyze(df)
user_report.show_html('user_report.html')

                                             |                                             | [  0%]   00:00 ->…

PAIRWISE CALCULATION LENGTH WARNING: There are 254 features in this dataframe and the 'pairwise_analysis' parameter is set to 'auto'.
Pairwise analysis is exponential in length: 254 features will cause ~64516 pairs to be evaluated, which could take a long time.

You must call the function with the parameter pairwise_analysis='on' or 'off' to explicitly select desired behavior.


AttributeError: 'DataframeReport' object has no attribute 'associations_html_source'

## All Formula in an object

In [3]:
import requests
import pandas as pd
import keyring

def get_salesforce_formulas(object_name):
    # Retrieve your credentials
    username = keyring.get_password("salesforce", "username")
    password = keyring.get_password("salesforce", "password")
    security_token = keyring.get_password("salesforce", "security_token")
    consumer_key = keyring.get_password("salesforce", "consumer_key")
    consumer_secret = keyring.get_password("salesforce", "consumer_secret")

    # Step 1: Obtain OAuth 2.0 token
    token_url = "https://login.salesforce.com/services/oauth2/token"
    payload = {
        'grant_type': 'password',
        'client_id': consumer_key,
        'client_secret': consumer_secret,
        'username': username,
        'password': password + security_token
    }
    response = requests.post(token_url, data=payload)
    response.raise_for_status()  # Check if the request was successful

    # Extract access token from the response
    access_token = response.json().get('access_token')
    instance_url = response.json().get('instance_url')

    # Step 2: Retrieve object metadata
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/json'
    }
    metadata_url = f"{instance_url}/services/data/v52.0/sobjects/{object_name}/describe"
    metadata_response = requests.get(metadata_url, headers=headers)
    metadata_response.raise_for_status()  # Check if the request was successful
    metadata = metadata_response.json()

    # Step 3: Extract formulas for custom fields
    formulas = []
    for field in metadata['fields']:
        if field.get('calculated'):
            formulas.append({
                'Field Name': field['name'],
                'Formula': field.get('calculatedFormula')
            })

    # Convert the list of formulas to a pandas DataFrame
    df = pd.DataFrame(formulas)
    return df

# Example usage
object_name = 'User'
formulas_df = get_salesforce_formulas(object_name)
formulas_df


,Field Name,Formula
0,License_Type__c,Profile.UserLicense.Name
1,lightico__Profile_Name__c,"IF(TRIM(Profile.Name) = 'PT1', 'System Adminis..."
2,lightico__Role_Name__c,UserRole.Name
3,Profile_Dev_Name__c,Profile.Name
4,Role_Dev_Name__c,UserRole.Name
5,Viewing_Offices_Text__c,"Text(POD__c) &"";""&Text(Office__c)&"";""&\r\nIF (..."
6,Staff_Record__c,"IF(ISBLANK(Staff_Record_ID__c),""Building Staff..."


In [6]:
formulas_df.to_excel('user_formula.xlsx')

## Formula for specific Field

In [9]:
import requests
import keyring

def get_salesforce_formula(object_name, field_name):
    # Retrieve your credentials
    username = keyring.get_password("salesforce", "username")
    password = keyring.get_password("salesforce", "password")
    security_token = keyring.get_password("salesforce", "security_token")
    consumer_key = keyring.get_password("salesforce", "consumer_key")
    consumer_secret = keyring.get_password("salesforce", "consumer_secret")

    # Step 1: Obtain OAuth 2.0 token
    token_url = "https://login.salesforce.com/services/oauth2/token"
    payload = {
        'grant_type': 'password',
        'client_id': consumer_key,
        'client_secret': consumer_secret,
        'username': username,
        'password': password + security_token
    }
    response = requests.post(token_url, data=payload)
    response.raise_for_status()  # Check if the request was successful

    # Extract access token from the response
    access_token = response.json().get('access_token')
    instance_url = response.json().get('instance_url')

    # Step 2: Retrieve object metadata
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/json'
    }
    metadata_url = f"{instance_url}/services/data/v52.0/sobjects/{object_name}/describe"
    metadata_response = requests.get(metadata_url, headers=headers)
    metadata_response.raise_for_status()  # Check if the request was successful
    metadata = metadata_response.json()

    # Step 3: Find the specific field and extract its formula
    formula = None
    for field in metadata['fields']:
        if field['name'] == field_name:
            if field.get('calculated') and field.get('calculatedFormula'):
                formula = field['calculatedFormula']
            else:
                formula = "The specified field is not a calculated field."
            break
    else:
        formula = "Field not found in the specified object."

    return formula

# Example usage
object_name = 'Contact'
field_name = 'Mortgage_Referred_By_Who__c'
formula = get_salesforce_formula(object_name, field_name)
print(f"Field: {field_name}, Formula: {formula}")


Field: Mortgage_Referred_By_Who__c, Formula: The specified field is not a calculated field.


In [16]:
import requests
import pandas as pd
from simple_salesforce import Salesforce
import keyring

def export_salesforce_data(query, export_file):
    # Retrieve your credentials
    username = keyring.get_password("salesforce", "username")
    password = keyring.get_password("salesforce", "password")
    security_token = keyring.get_password("salesforce", "security_token")
    consumer_key = keyring.get_password("salesforce", "consumer_key")
    consumer_secret = keyring.get_password("salesforce", "consumer_secret")

    # Step 1: Obtain OAuth 2.0 token
    token_url = "https://login.salesforce.com/services/oauth2/token"
    payload = {
        'grant_type': 'password',
        'client_id': consumer_key,
        'client_secret': consumer_secret,
        'username': username,
        'password': password + security_token
    }
    response = requests.post(token_url, data=payload)
    response.raise_for_status()  # Check if the request was successful

    # Extract access token from the response
    access_token = response.json().get('access_token')
    instance_url = response.json().get('instance_url')

    # Step 2: Authenticate to Salesforce using the access token
    sf = Salesforce(instance_url=instance_url, session_id=access_token)

    # Step 3: Query data
    records = []
    query_result = sf.query_all(query)
    records.extend(query_result['records'])

    # Continue querying if there are more records
    while not query_result['done']:
        query_result = sf.query_more(query_result['nextRecordsUrl'], True)
        records.extend(query_result['records'])

    # Function to flatten nested dictionaries
    def flatten_record(record, parent_key='', sep='.'):
        items = []
        for k, v in record.items():
            new_key = f"{parent_key}{sep}{k}" if parent_key else k
            if isinstance(v, dict):
                items.extend(flatten_record(v, new_key, sep=sep).items())
            else:
                items.append((new_key, v))
        return dict(items)

    # Flatten all records
    flattened_records = [flatten_record(record) for record in records]

    # Convert the data to a pandas DataFrame
    df = pd.DataFrame(flattened_records)

    # Clean up the DataFrame (remove Salesforce metadata)
    if 'attributes.type' in df.columns:
        df = df.drop(columns=['attributes.type', 'attributes.url'])

    # Display the DataFrame
    return df

# Example usage
query = """
SELECT
    User__c, 
    Name, 
    Primary__r.Name,
    Primary__r.POD__r.Name,
    Role_Title__r.Name  
FROM
    Staff__c
"""
export_file = 'test.csv'

# Get the DataFrame
df = export_salesforce_data(query, export_file)

# Display the DataFrame
df

# Optionally, export the DataFrame to a CSV file
df.to_csv(export_file, index=False)
print(f"Data exported to {export_file}")


Data exported to test.csv


In [14]:
df

,User__c,Name,Primary__r.attributes.type,Primary__r.attributes.url,Primary__r.Name,Primary__r.POD__r.attributes.type,Primary__r.POD__r.attributes.url,Primary__r.POD__r.Name,Role_Title__r.attributes.type,Role_Title__r.attributes.url,Role_Title__r.Name,Primary__r.POD__r,Role_Title__r
0,0058W00000EKrpxQAD,Mehak Nazam,pba__Office__c,/services/data/v59.0/sobjects/pba__Office__c/a...,Financial Services,POD_Grouping__c,/services/data/v59.0/sobjects/POD_Grouping__c/...,Financial Services,Role__c,/services/data/v59.0/sobjects/Role__c/a4m8W000...,Relationship Manager,NaN,NaN
1,0058W00000EKsUHQA1,Lindsay Heath,pba__Office__c,/services/data/v59.0/sobjects/pba__Office__c/a...,Financial Services,POD_Grouping__c,/services/data/v59.0/sobjects/POD_Grouping__c/...,Financial Services,Role__c,/services/data/v59.0/sobjects/Role__c/a4m8W000...,Mortgage & Protection Advisor,NaN,NaN
2,0058W00000ELDWxQAP,Brooke Rushworth,pba__Office__c,/services/data/v59.0/sobjects/pba__Office__c/a...,To Be Completed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0058W00000ELWtIQAX,Christian Vose,pba__Office__c,/services/data/v59.0/sobjects/pba__Office__c/a...,Droylsden,POD_Grouping__c,/services/data/v59.0/sobjects/POD_Grouping__c/...,Droylsden & Gorton,Role__c,/services/data/v59.0/sobjects/Role__c/a4m8W000...,Sales Progressor,NaN,NaN
4,0058W00000ELYo4QAH,Issabella Chapman,pba__Office__c,/services/data/v59.0/sobjects/pba__Office__c/a...,Auction,POD_Grouping__c,/services/data/v59.0/sobjects/POD_Grouping__c/...,Auction,Role__c,/services/data/v59.0/sobjects/Role__c/a4m8W000...,Auction Consultant,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,0058W00000GGiruQAD,Megan Edwards,pba__Office__c,/services/data/v59.0/sobjects/pba__Office__c/a...,Stockton Heath,POD_Grouping__c,/services/data/v59.0/sobjects/POD_Grouping__c/...,Cheshire (Stockton Heath & Northwich),NaN,NaN,NaN,NaN,NaN
157,0058W00000GHExkQAH,Deepankar Srigyan,pba__Office__c,/services/data/v59.0/sobjects/pba__Office__c/a...,Head Office,POD_Grouping__c,/services/data/v59.0/sobjects/POD_Grouping__c/...,Head Office,Role__c,/services/data/v59.0/sobjects/Role__c/a4m8W000...,Data Analyst,NaN,NaN
158,0058W00000GHGGYQA5,Grace Beaumont,pba__Office__c,/services/data/v59.0/sobjects/pba__Office__c/a...,Head Office,POD_Grouping__c,/services/data/v59.0/sobjects/POD_Grouping__c/...,Head Office,Role__c,/services/data/v59.0/sobjects/Role__c/a4m8W000...,Group Operations Manager,NaN,NaN
159,None,Catherine Dixon,pba__Office__c,/services/data/v59.0/sobjects/pba__Office__c/a...,Head Office,POD_Grouping__c,/services/data/v59.0/sobjects/POD_Grouping__c/...,Head Office,NaN,NaN,NaN,NaN,NaN
